# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 94 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2304.00037


extracting tarball to tmp_2304.00037...

 done.
Retrieving document from  https://arxiv.org/e-print/2304.00267


extracting tarball to tmp_2304.00267...

 done.


list index out of range
Retrieving document from  https://arxiv.org/e-print/2304.00461


extracting tarball to tmp_2304.00461...

 done.


/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.16/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3505: LatexWarning: Found documentclass in tmp_2304.00461/paper_final.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.00267-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.00267) | **Digging into the Interior of Hot Cores with ALMA (DIHCA). III: The  Chemical Link between NH$_{2}$CHO, HNCO, and H$_{2}$CO**  |
|| K. Taniguchi, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-04-04*|
|*Comments*| *Accepted for The Astrophysical Journal. 27 pages, 10 tables, and 13 figures*|
|**Abstract**| We have analyzed the NH$_{2}$CHO, HNCO, H$_{2}$CO, and CH$_{3}$CN ($^{13}$CH$_{3}$CN) molecular lines at an angular resolution of $\sim 0.3''$ obtained by the Atacama Large Millimeter/submillimeter Array (ALMA) Band 6 toward 30 high-mass star-forming regions. The NH$_{2}$CHO emission has been detected in 23 regions, while the other species have been detected toward 29 regions. A total of 44 hot molecular cores (HMCs) have been identified using the moment 0 maps of the CH$_{3}$CN line. The fractional abundances of the four species have been derived at each HMC. In order to investigate pure chemical relationships, we have conducted a partial correlation test to exclude the effect of temperature. Strong positive correlations between NH$_{2}$CHO and HNCO ($\rho=0.89$) and between NH$_{2}$CHO and H$_{2}$CO (0.84) have been found. These strong correlations indicate their direct chemical links; dual-cyclic hydrogen addition and abstraction reactions between HNCO and NH$_{2}$CHO and gas-phase formation of NH$_{2}$CHO from H$_{2}$CO. Chemical models including these reactions can reproduce the observed abundances in our target sources. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.00037-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.00037) | **A 1.3% distance to M33 from HST Cepheid photometry**  |
|| L. Breuval, et al. -- incl., <mark>S. Li</mark> |
|*Appeared on*| *2023-04-04*|
|*Comments*| *Submitted to ApJ, comments welcome*|
|**Abstract**| We present a low-dispersion period-luminosity relation (PL) based on 154 Cepheids in Messier 33 (M33) with Hubble Space Telescope (HST) photometry from the PHATTER survey. Using high-quality ground-based light curves, we recover Cepheid phases and amplitudes for multi-epoch HST data and we perform template fitting to derive intensity-averaged mean magnitudes. HST observations in the SH0ES near-infrared Wesenheit system significantly reduce the effect of crowding relative to ground-based data, as seen in the final PL scatter of $\sigma$ = 0.11 mag. We adopt the absolute calibration of the PL based on HST observations in the Large Magellanic Cloud (LMC) and a distance derived using late-type detached eclipsing binaries to obtain a distance modulus for M33 of $\mu$ = 24.622 $\pm$ 0.030 mag (d = 840 $\pm$ 11 kpc), a best-to-date precision of 1.3%. We find very good agreement with past Cepheid-based measurements. Several TRGB estimates bracket our result while disagreeing with each other. Finally, we show that the flux contribution from star clusters hosting Cepheids in M33 does not impact the distance measurement and we find only 3.7% of the sample is located in (or nearby) young clusters. M33 offers one of the best sites for the cross-calibration of many primary distance indicators. Thus, a precise independent geometric determination of its distance would provide a valuable new anchor to measure the Hubble constant. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2304.00461-b31b1b.svg)](https://arxiv.org/abs/arXiv:2304.00461) | **Detection of TiO and VO in the atmosphere of WASP-121b and Evidence for  its temporal variation**  |
|| Q. Ouyang, et al. -- incl., <mark>M. Zhai</mark> |
|*Appeared on*| *2023-04-04*|
|*Comments*| *23 pages, 13 figures, 6 tables. Accepted for publication in Research in Astronomy and Astrophysics*|
|**Abstract**| We report the transit observations of the ultra hot Jupiter WASP-121b using the Goodman High Throughput Spectrograph (GHTS) at the 4-meter ground-based telescope Southern Astrophysical Research Telescope (SOAR), covering the wavelength range $502-900$ nm. By dividing the target and reference star into 19 spectroscopic passbands and applying differential spectrophotometry, we derive spectroscopic transit light curves and fit them using Gaussian process framework to determine transit depths for every passbands. The obtained optical transmission spectrum shows a steep increased slope toward the blue wavelength, which seems to be too steep to be accounted for by the Rayleigh scattering alone. We note that the transmission spectrum from this work and other works differ obviously from each other, which was pointed out previously by \citet{Wilson2021} as evidence for temporal atmospheric variation. We perform a free chemistry retrieval analysis on the optical transmission spectra from this work and the literature HST/WFC3 NIR spectrum. We determine TiO, VO and H$_{2}$O with abundances of $-5.95_{-0.42}^{+0.47}$ dex, $-6.72_{-1.79}^{+0.51}$ dex, and $-4.13_{-0.46}^{+0.63}$ dex, respectively. We compare the abundances of all these three molecules derived from this work and previous works, and find that they are not consistent with each other, indicating the chemical compositions of the terminator region may change over long timescales. Future multi-epoch and high-precision transit observations are required to further confirm this phenomena. We note that when combining the transmission spectra in the optical and in NIR in retrieval analysis, the abundances of V and VO, the NIR-to-optical offset and the cloud deck pressure may be coupled with each other. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2304.00267.md
    + _build/html/tmp_2304.00267/./fig_modelBfinal.png
    + _build/html/tmp_2304.00267/./fig_mom01.png
    + _build/html/tmp_2304.00267/./fig_mom04.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Digging into the Interior of Hot Cores with ALMA (DIHCA). III: \\The Chemical Link between NH$_{2}$CHO, HNCO, and H$_{2}$CO

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2304.00267-b31b1b.svg)](https://arxiv.org/abs/2304.00267)<mark>Appeared on: 2023-04-04</mark> -  _Accepted for The Astrophysical Journal. 27 pages, 10 tables, and 13 figures_

</div>
<div id="authors">

K. Taniguchi, et al. -- incl., <mark>S. Li</mark>

</div>
<div id="abstract">

**Abstract:** We have analyzed the NH $_{2}$ CHO, HNCO, H $_{2}$ CO, and CH $_{3}$ CN ( $^{13}$ CH $_{3}$ CN) molecular lines at an angular resolution of $\sim 0\farcs3$ obtained by the Atacama Large Millimeter/submillimeter Array (ALMA) Band 6 toward 30 high-mass star-forming regions.The NH $_{2}$ CHO emission has been detected in 23 regions, while the other species have been detected toward 29 regions.A total of 44 hot molecular cores (HMCs) have been identified using the moment 0 maps of the CH $_{3}$ CN line.The fractional abundances of the four species have been derived at each HMC.In order to investigate pure chemical relationships, we have conducted a partial correlation test to exclude the effect of temperature.Strong positive correlations between NH $_{2}$ CHO and HNCO ( $\rho=0.89$ ) and between NH $_{2}$ CHO and H $_{2}$ CO (0.84) have been found.These strong correlations indicate their direct chemical links; dual-cyclic hydrogen addition and abstraction reactions between HNCO and NH $_{2}$ CHO and gas-phase formation of NH $_{2}$ CHO from H $_{2}$ CO.Chemical models including these reactions can reproduce the observed abundances in our target sources.

</div>

<div id="div_fig1">

<img src="tmp_2304.00267/./fig_modelBfinal.png" alt="Fig8" width="100%"/>

**Figure 8. -** Comparison with Model B \citep{2020ApJ...895...86G}. Panels from top to bottom show results of HNCO, NH$_{2}$CHO, H$_{2}$CO, and CH$_{3}$CN, respectively. The modeled abundances plotted here are the values at the end of the simulations; $t$ = $t_{\rm{collapse}}$ + $t_{\rm{warm-up}}$($5\times10^4$ yrs)+ $t_{\rm{post warm-up}}$($10^5$ yrs). Left panels show dependences on different collapsing timescale and maximum density with the maximum temperature of 200 K. Right panels show dependences on different collapsing timescale and maximum temperature with the maximum density of $10^7$ cm$^{-3}$. The four representative observed abundances are plotted (Maximum, Average, Median, and Minimum). The blue filled ranges indicate the ranges between average and median values.  (*fig:modelB*)

</div>
<div id="div_fig2">

<img src="tmp_2304.00267/./fig_mom01.png" alt="Fig9" width="100%"/>

**Figure 9. -** Continuum images (gray scales) overlaid with contours indicating moment 0 maps of molecular lines (left panels: white; CH$_{3}$CN and cyan; H$_{2}$CO, right panels: magenta; NH$_{2}$CHO and yellow; HNCO). Red crosses indicate positions of hot molecular cores (HMCs). Information on noise levels and contour levels are summarized in Tables \ref{tab:mom0info} and \ref{tab:mom0info2}.  (*fig:mom01*)

</div>
<div id="div_fig3">

<img src="tmp_2304.00267/./fig_mom04.png" alt="Fig12" width="100%"/>

**Figure 12. -** Continued. (*fig:mom04*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2304.00267"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

182

  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
